In [173]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler 
import time
import warnings

In [174]:
from autoimpute.imputations import MiceImputer,SingleImputer

In [175]:
from sklearn.impute import SimpleImputer

In [176]:
from statsmodels.imputation.mice import MICE
from statsmodels.imputation.mice import MICEData
import statsmodels.api as sm

In [177]:
# Set some debugging parameters
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

In [178]:
TRAINING_PERIOD = 4
LAG_PREDICTORS = True
NLAG = 1
DEPVAR = "asylum seekers" 
# "asylum seekers" | "asylum seekers (global stock)" | "refugees (global stock)"
LOG_TRANSFORM_DEPVAR = True
PER_CAPITA_TRANSFORM_DEPVAR = True
SPEI_MOVING_AVERAGES_3_6 = False
NSTEP_AHEAD = 1
CALCULATE_INTERACTIONS = False

In [179]:
# Read Dataset directly from file and pre-process
original_dataset = pd.read_csv("../Data/replication_data.csv")
original_dataset['n_asylum'] = original_dataset['n_asylum'].fillna(0)
original_dataset['n_asylum_eu28'] = original_dataset['n_asylum_eu28'].fillna(0)
original_dataset['n_refugees'] = original_dataset['n_refugees'].fillna(0)
original_dataset['spei3_gs_neg'] = original_dataset['spei3_gs_neg'].apply(lambda x: x*12)
original_dataset['spei3_gs_pos'] = original_dataset['spei3_gs_pos'].apply(lambda x: x*12)
original_dataset['physical_integrity'] = original_dataset['civil_liberties_combined']

In [180]:
# Remove all microstates from the list
microstates = pd.read_csv("../Data/microstates.dat", sep = "\t", names=['gwcode','iso3','name','d1','d2'])
microstates_list = microstates['gwcode'].to_list()
dataset = original_dataset[~original_dataset['gwcode'].isin(microstates_list)]

In [181]:
# Original code did log(population+1),  I have directly did log population and inf values are 0
if DEPVAR == "asylum seekers":
    dataset['orig_depvar'] = dataset['n_asylum_eu28']
    dataset['ln_depvar_pop'] = dataset['n_asylum_eu28']
    dataset.loc[dataset['n_asylum_eu28']>0, 'ln_depvar_pop'] = dataset['n_asylum_eu28']/(dataset['wdi_pop']/1e6)
    dataset.loc[dataset['n_asylum_eu28']<=0, 'ln_depvar_pop'] = 1
    dataset['ln_depvar_pop'] = pd.to_numeric(dataset['ln_depvar_pop'])
    dataset['ln_depvar_pop'] = dataset['ln_depvar_pop'].apply(lambda x: np.log(x))

# Has to be changed depending on what we are predicting
dataset['depvar'] = dataset['ln_depvar_pop']


In [182]:
impute_dataset = dataset[['country', 'year', 'depvar', 'highest_neighbor_dem', 'area', 'wdi_pop', 'wdi_urban_pop', 'distance_to_eu',\
                          'wdi_gdppc_growth', 'wdi_gdppc', 'perc_post_secondary', 'kof_index', 'wdi_imr',\
                          'tmp_pop', 'spei3_gs_pos', 'spei3_gs_neg', \
                          'casualties_brd', 'annually_affected_20k', 'physical_integrity', 'free_movement', 'homicide']]

In [183]:
impute_dataset['wdi_pop'] = impute_dataset['wdi_pop'].apply(lambda x: np.log(x))
impute_dataset['area'] = impute_dataset['area'].apply(lambda x: np.log(x))
impute_dataset['wdi_gdppc'] = impute_dataset['wdi_gdppc'].apply(lambda x: np.log(x))
impute_dataset['casualties_brd'] = impute_dataset['casualties_brd'].apply(lambda x: np.log(x+1))
impute_dataset['annually_affected_20k'] = impute_dataset['annually_affected_20k'].apply(lambda x: np.log(x+1))
impute_dataset['homicide'] = impute_dataset['homicide'].apply(lambda x: np.log(x+1))

In [184]:
# Second order tmp_pop
impute_dataset['tmp_pop_sq'] = impute_dataset['tmp_pop'].apply(lambda x: np.power(x,2))

# impute_dataset['spei3_gs_pos'] = -1*impute_dataset['spei3_gs_pos']
# impute_dataset['spei3_gs_neg'] = -1*impute_dataset['spei3_gs_neg']

# 3 year moving average
impute_dataset['spei3_gs_pos_r3'] = impute_dataset['spei3_gs_pos'].rolling(3,min_periods=1).mean()
impute_dataset['spei3_gs_neg_r3'] = impute_dataset['spei3_gs_neg'].rolling(3,min_periods=1).mean()
# 6 year moving average
# impute_dataset['spei3_gs_pos_r6'] = impute_dataset['spei3_gs_pos'].rolling(6,min_periods=1).mean()
# impute_dataset['spei3_gs_neg_r6'] = impute_dataset['spei3_gs_neg'].rolling(6,min_periods=1).mean()


In [185]:
# Filter for only years we want
impute_dataset = impute_dataset[impute_dataset['year']>=1998]

### Imputations are calculated here:
<ol>
    <li> Naive Imputation </li>
    <li> AutoImpute </li>
    <li> Statsmodels </li>
</ol>

In [186]:
# Global imputation parameters
n_imputations = 10 # number of imputations
impute_dataset = impute_dataset.replace([np.inf, -np.inf], np.nan).reset_index()
imputations = []
print(impute_dataset)

      index      country  year    depvar  highest_neighbor_dem       area  \
0         6       guyana  1998  0.000000                 0.857  12.190197   
1         7       guyana  1999  0.288117                 0.883  12.190197   
2         8       guyana  2000  0.000000                 0.876  12.190197   
3         9       guyana  2001  0.294095                 0.864  12.190197   
4        10       guyana  2002  4.543150                 0.851  12.190197   
...     ...          ...   ...       ...                   ...        ...   
3608   5081  south sudan  2014  2.248582                 0.457        NaN   
3609   5082  south sudan  2015  2.390468                 0.463        NaN   
3610   5083  south sudan  2016  2.711198                 0.504        NaN   
3611   5084  south sudan  2017  2.775037                 0.465        NaN   
3612   5085  south sudan  2018  3.597010                 0.447        NaN   

        wdi_pop  wdi_urban_pop  distance_to_eu  wdi_gdppc_growth  wdi_gdppc

In [142]:
# Naive Impute
impute_dataset = impute_dataset.fillna(0)

In [143]:
#AutoImpute


In [187]:
c = impute_dataset['country']
y = impute_dataset['year']
impute_dataset = impute_dataset.drop(columns=['country','year'])
imp = MICEData(impute_dataset)
fml = 'depvar ~ highest_neighbor_dem+ area+ wdi_pop+ wdi_urban_pop+ distance_to_eu+ wdi_gdppc_growth+ wdi_gdppc+ perc_post_secondary+ kof_index+ wdi_imr+tmp_pop+ spei3_gs_pos+ spei3_gs_neg+casualties_brd+ annually_affected_20k+ physical_integrity+ free_movement+ homicide'
mice = MICE(fml, sm.OLS, imp)
results = mice.fit(3, n_imputations)
for j in range(n_imputations):
    print(j)
    imp.update_all()
    temp = imp.data
    temp['country'] = c
    temp['year'] = y

    temp.loc[temp['spei3_gs_neg']<=0, 'spei3_gs_neg'] = 0
    temp.loc[temp['spei3_gs_pos']<=0, 'spei3_gs_pos'] = 0
    temp.loc[temp['kof_index']<=0, 'kof_index'] = 0
    temp.loc[temp['wdi_urban_pop']<=0, 'wdi_urban_pop'] = 0
    temp.loc[temp['perc_post_secondary']<=0, 'perc_post_secondary'] = 0
    temp.loc[temp['wdi_imr']<=0, 'wdi_imr'] = 0
    temp.loc[temp['distance_to_eu']<=0, 'distance_to_eu'] = 0

    imputations.append(temp)
    
# impute_dataset = imputations[0]
# print(impute_dataset)

0
1
2
3
4
5
6
7
8
9


In [188]:
# grouped_countries = impute_dataset.groupby(['country'])
# # Lag the dataset
# lagged_frame_original = pd.DataFrame()
# for country, feats in grouped_countries:
#     features = grouped_countries.get_group(country).sort_values(by='year')#.drop(columns=['country','year','depvar'])
#     features['depvar'] = features['depvar'].shift(-1)
#     features = features[:-1] # to lag 
#     lagged_frame_original = pd.concat([lagged_frame_original,features])

KeyError: 'country'

In [ ]:
#Construct year slices -  as this is predictor lag - 1998 predicts 1999
year_range = [1998,2017]
window = 4
step_ahead = 1
slices = []
for i in range(year_range[0],year_range[1]-window+1):
    slices.append(([j for j in range(i,i+window)],i+window-1+step_ahead))

print(slices)
# slices = slices[:-1]


In [ ]:
baseline_columns = ['highest_neighbor_dem', 'area', 'wdi_pop', 'wdi_urban_pop', 'distance_to_eu']
economy_columns = ['wdi_gdppc_growth', 'wdi_gdppc', 'perc_post_secondary', 'kof_index', 'wdi_imr']
climate_columns = ['tmp_pop', 'spei3_gs_pos', 'spei3_gs_neg','spei3_gs_pos_r3','spei3_gs_neg_r3','tmp_pop_sq']
violence_columns = ['casualties_brd', 'annually_affected_20k', 'physical_integrity', 'free_movement', 'homicide']

In [ ]:
to_drop = [
    economy_columns+climate_columns,  #only violence+baseline
    economy_columns+violence_columns, #only climate+baseline
    climate_columns+violence_columns, #only economy+baseline
    [],                               #all
    economy_columns+violence_columns+climate_columns #only baseline
]

In [189]:
for impute_dataset in imputations:
    grouped_countries = impute_dataset.groupby(['country'])
    # Lag the dataset
    lagged_frame_original = pd.DataFrame()
    for country, feats in grouped_countries:
        features = grouped_countries.get_group(country).sort_values(by='year')#.drop(columns=['country','year','depvar'])
        features['depvar'] = features['depvar'].shift(-1)
        features = features[:-1] # to lag 
        lagged_frame_original = pd.concat([lagged_frame_original,features])
    
    avg_tot_mae = 0
    for dropped in to_drop:
        avg_mae = 0
        for idx in range(len(slices)):
            s = slices[idx]
            print(idx,"/",len(slices))
            lagged_frame = lagged_frame_original.drop(columns=dropped)

            train_set = lagged_frame.loc[lagged_frame['year'].isin(s[0])]
            train_features = train_set.drop(columns=['country','year','depvar']).to_numpy()
            train_targets = train_set['depvar'].to_numpy()
            scaler_train = StandardScaler(with_mean=True,with_std=True)
            train_features = scaler_train.fit_transform(train_features,train_targets)

            test_set = lagged_frame.loc[lagged_frame['year'] == s[1]]
            test_features = test_set.drop(columns=['country','year','depvar']).to_numpy()
            test_targets = test_set['depvar'].to_numpy()
            scaler_test = StandardScaler(with_mean=True,with_std=True)
            test_features = scaler_test.fit_transform(test_features,test_targets)

            clf = RandomForestRegressor(n_estimators=1000, min_samples_split=5)
            clf.fit(train_features,train_targets)
            predictions = clf.predict(test_features)
            new_pred = []
            new_test = []
            cnt=0
            for i in range(len(predictions)):
                new_pred.append(predictions[i])
                new_test.append(test_targets[i])
            mae = mean_absolute_error(new_test,new_pred)
            avg_mae+=mae
            print("MAE for this run: ",mae)
        avg_tot_mae += avg_mae
        print("Average MAE for this imputation: ",avg_mae/len(slices))
    print("Average MAE for all imputations:",avg_tot_mae/len(imputations))


0 / 16
MAE for this run:  0.6707360419839231
1 / 16
MAE for this run:  0.5908677389014184
2 / 16
MAE for this run:  0.5781990874696876
3 / 16
MAE for this run:  0.6826895407662253
4 / 16
MAE for this run:  0.625622344120744
5 / 16
MAE for this run:  0.6799294904103572
6 / 16
MAE for this run:  0.6361978650839633
7 / 16
MAE for this run:  0.6204301568781087
8 / 16
MAE for this run:  0.6498651960267954
9 / 16
MAE for this run:  0.5918893143468164
10 / 16
MAE for this run:  0.5651014973188373
11 / 16
MAE for this run:  0.7528017128352935
12 / 16
MAE for this run:  0.7102639331311416
13 / 16
MAE for this run:  0.7097320404417335
14 / 16
MAE for this run:  0.6712463424122805
15 / 16
MAE for this run:  1.085411940056973
Average MAE for this imputation:  0.6763115151365187
0 / 16
MAE for this run:  0.8039960818354308
1 / 16
MAE for this run:  0.7619449029315637
2 / 16
MAE for this run:  0.7278185872131099
3 / 16
MAE for this run:  0.7667673472955083
4 / 16
MAE for this run:  0.786514160568277

MAE for this run:  0.644138949941701
9 / 16
MAE for this run:  0.5939081266036486
10 / 16
MAE for this run:  0.5686942738568848
11 / 16
MAE for this run:  0.7507149987884335
12 / 16
MAE for this run:  0.7101369151359191
13 / 16
MAE for this run:  0.7047028013281816
14 / 16
MAE for this run:  0.6702715158218164
15 / 16
MAE for this run:  1.0867259697846665
Average MAE for this imputation:  0.6767010250212974
0 / 16
MAE for this run:  0.802579054170668
1 / 16
MAE for this run:  0.7623452225079149
2 / 16
MAE for this run:  0.7265397853902331
3 / 16
MAE for this run:  0.7662500023416969
4 / 16
MAE for this run:  0.7839812341661644
5 / 16
MAE for this run:  0.8395693006860168
6 / 16
MAE for this run:  0.7816295358334262
7 / 16
MAE for this run:  0.695945070480253
8 / 16
MAE for this run:  0.7670843503551454
9 / 16


KeyboardInterrupt: 